In [1]:
# Import SparkSession
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder.appName("OnlineRetailEDA").getOrCreate()

# Load datasets
preprocessed_df = spark.read.csv("s3://project-tarun/Ecommerce_Final_Cleaned.csv", header=True, inferSchema=True)
raw_df = spark.read.csv("s3://project-tarun/OnlineRetail.csv", header=True, inferSchema=True)

# Optional: Clean raw_df column names (remove spaces if needed)
from pyspark.sql.functions import col

# Selecting original (unscaled) Quantity and UnitPrice from raw_df
raw_trimmed = raw_df.select(
    col("InvoiceNo").alias("InvoiceNo_raw"),
    col("StockCode").alias("StockCode_raw"),
    col("Quantity").alias("Quantity_raw"),
    col("UnitPrice").alias("UnitPrice_raw")
)

# Join raw and preprocessed data on InvoiceNo and StockCode
df = preprocessed_df.join(
    raw_trimmed,
    (preprocessed_df.InvoiceNo == raw_trimmed.InvoiceNo_raw) &
    (preprocessed_df.StockCode == raw_trimmed.StockCode_raw),
    how="inner"
)

# Replace normalized values with raw ones
df = df.drop("Quantity", "UnitPrice") \
       .withColumnRenamed("Quantity_raw", "Quantity") \
       .withColumnRenamed("UnitPrice_raw", "UnitPrice")

# Optional: Drop join keys if not needed
df = df.drop("InvoiceNo_raw", "StockCode_raw")

# Show schema and sample rows
df.printSchema()
df.show(5, truncate=False)


VBox()

Starting Spark application


ID,Kind,State,Spark UI,Driver log,User,Current session?
0,pyspark,idle,Link,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)

+---------+---------+---------------------------------+--------------+----------+--------------+--------+---------+
|InvoiceNo|StockCode|Description                      |InvoiceDate   |CustomerID|Country       |Quantity|UnitPrice|
+---------+---------+---------------------------------+--------------+----------+--------------+--------+---------+
|536373   |21068    |VINTAGE BILLBOARD LOVE/HATE MUG  |12/1/2010 9:02|17850     |United Kingdom|6       |1.06     |
|536375   |21730    |GLASS STAR FROSTED T-LIGHT HOLDER|12/1/2010 9:32|17850     |United Kingdom|6       |4.25     |
|536375   |84029E   |RED WOOLLY HOTTIE WHITE HEART.   |12/1/2010 9:32|17850     |U

In [2]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql.functions import to_timestamp, coalesce

df = df.withColumn(
    "InvoiceTimestamp",
    coalesce(
        to_timestamp("InvoiceDate", "MM/dd/yyyy HH:mm"),
        to_timestamp("InvoiceDate", "MM-dd-yyyy HH:mm:ss")
    )
)
df = df.drop("InvoiceDate")  # Drop InvoiceDate
df.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+--------------------+----------+--------------+--------+---------+-------------------+
|InvoiceNo|StockCode|         Description|CustomerID|       Country|Quantity|UnitPrice|   InvoiceTimestamp|
+---------+---------+--------------------+----------+--------------+--------+---------+-------------------+
|   536373|    21068|VINTAGE BILLBOARD...|     17850|United Kingdom|       6|     1.06|2010-12-01 09:02:00|
|   536375|    21730|GLASS STAR FROSTE...|     17850|United Kingdom|       6|     4.25|2010-12-01 09:32:00|
|   536375|   84029E|RED WOOLLY HOTTIE...|     17850|United Kingdom|       6|     3.39|2010-12-01 09:32:00|
|   536378|   84997B|RED 3 PIECE RETRO...|     14688|United Kingdom|      12|     3.75|2010-12-01 09:37:00|
|   536381|   37444A|YELLOW BREAKFAST ...|     15311|United Kingdom|       1|     2.95|2010-12-01 09:41:00|
|   536384|    22470|HEART OF WICKER L...|     18074|United Kingdom|      40|     2.55|2010-12-01 09:53:00|
|   536384|    82484|WOOD BL

EXPLORATORY DATA ANALYSIS (EDA)

Count of Customers per Country

In [7]:
from pyspark.sql import functions as F
# Count distinct CustomerID per Country
df.groupBy("Country").agg(F.countDistinct("CustomerID").alias("UniqueCustomers")) \
  .orderBy("UniqueCustomers", ascending=False) \
  .show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+---------------+
|        Country|UniqueCustomers|
+---------------+---------------+
| United Kingdom|           3920|
|        Germany|             94|
|         France|             88|
|          Spain|             30|
|        Belgium|             25|
|    Switzerland|             22|
|       Portugal|             20|
|          Italy|             14|
|        Finland|             12|
|        Austria|             11|
|         Norway|             10|
|    Netherlands|              9|
|        Denmark|              9|
|Channel Islands|              9|
|      Australia|              9|
|         Cyprus|              8|
|          Japan|              8|
|         Sweden|              8|
|         Poland|              6|
|    Unspecified|              5|
+---------------+---------------+
only showing top 20 rows

Total Quantity Sold per Country

In [8]:
# Sum quantity per country
df.groupBy("Country").agg(F.sum("Quantity").alias("TotalQuantity")) \
  .orderBy("TotalQuantity", ascending=False) \
  .show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------+
|        Country|TotalQuantity|
+---------------+-------------+
| United Kingdom|      4666030|
|    Netherlands|       199629|
|           EIRE|       146378|
|        Germany|       118739|
|         France|       112019|
|      Australia|        82277|
|         Sweden|        35795|
|    Switzerland|        30685|
|          Spain|        27859|
|          Japan|        25692|
|        Belgium|        23182|
|         Norway|        19260|
|       Portugal|        16331|
|        Finland|        11224|
|Channel Islands|         9421|
|        Denmark|         8288|
|          Italy|         7568|
|         Cyprus|         6305|
|      Singapore|         5249|
|      Hong Kong|         5015|
+---------------+-------------+
only showing top 20 rows

Total Revenue per Country

In [9]:
from pyspark.sql import functions as F

# First, create a new column for Revenue = Quantity * UnitPrice
df = df.withColumn("Revenue", F.col("Quantity") * F.col("UnitPrice"))

# Sum revenue per country
df.groupBy("Country").agg(F.sum("Revenue").alias("TotalRevenue")) \
  .orderBy("TotalRevenue", ascending=False) \
  .show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------+
|        Country|      TotalRevenue|
+---------------+------------------+
| United Kingdom| 8907239.845000258|
|           EIRE| 285647.4899999999|
|    Netherlands| 284225.4600000001|
|        Germany|230163.92000000004|
|         France|209690.90000000005|
|      Australia|         136824.03|
|          Spain| 61620.22000000001|
|    Switzerland|          57139.58|
|        Belgium|          41038.21|
|      Singapore|          39394.96|
|         Sweden| 38156.56999999999|
|         Norway| 37608.66000000001|
|          Japan|          36691.81|
|       Portugal|33842.149999999994|
|        Finland|22695.479999999996|
|Channel Islands|20310.440000000002|
|        Denmark|19038.630000000005|
|          Italy|17098.760000000002|
|      Hong Kong|16112.699999999999|
|         Cyprus|           13797.2|
+---------------+------------------+
only showing top 20 rows

Most Frequently Purchased Items

In [10]:
# Count frequency of each item description
df.groupBy("Description").agg(F.count("*").alias("Frequency")) \
  .orderBy("Frequency", ascending=False) \
  .show(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+
|         Description|Frequency|
+--------------------+---------+
|WHITE HANGING HEA...|     2446|
|JUMBO BAG RED RET...|     2157|
|REGENCY CAKESTAND...|     2062|
|       PARTY BUNTING|     1736|
|LUNCH BAG RED RET...|     1628|
|ASSORTED COLOUR B...|     1537|
|PACK OF 72 RETROS...|     1436|
|SET OF 3 CAKE TIN...|     1413|
|LUNCH BAG  BLACK ...|     1389|
|NATURAL SLATE HEA...|     1275|
+--------------------+---------+
only showing top 10 rows

Top Customers by Revenue

In [11]:
# Sum revenue per customer
df.groupBy("CustomerID").agg(F.sum("Revenue").alias("TotalRevenue")) \
  .orderBy("TotalRevenue", ascending=False) \
  .show(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+
|CustomerID|      TotalRevenue|
+----------+------------------+
|     16547|1772220.8299999991|
|     14646|278990.17999999993|
|     18102|          259657.3|
|     17450|194550.78999999998|
|     14911|143311.08999999997|
|     12415|         123182.69|
|     14156|         116999.43|
|     17511|           90958.7|
|     16029| 81482.76000000001|
|     12346|           77183.6|
+----------+------------------+
only showing top 10 rows

Number of Invoices per Customer

In [12]:
# Count distinct InvoiceNo per customer
df.groupBy("CustomerID").agg(F.countDistinct("InvoiceNo").alias("NumInvoices")) \
  .orderBy("NumInvoices", ascending=False) \
  .show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+
|CustomerID|NumInvoices|
+----------+-----------+
|     16547|       2189|
|     12748|        210|
|     14911|        201|
|     17841|        124|
|     13089|         97|
|     14606|         93|
|     15311|         91|
|     12971|         86|
|     14646|         74|
|     16029|         63|
|     13408|         62|
|     18102|         60|
|     13798|         57|
|     14527|         55|
|     14156|         55|
|     16422|         51|
|     13694|         50|
|     15061|         48|
|     16013|         47|
|     15039|         47|
+----------+-----------+
only showing top 20 rows

Monthly Revenue Trend

In [13]:
# Convert InvoiceTimestamp to a Month column
df = df.withColumn("Month", F.date_format("InvoiceTimestamp", "yyyy-MM"))

# Group by Month and calculate total revenue
df.groupBy("Month").agg(F.sum("Revenue").alias("MonthlyRevenue")) \
  .orderBy("Month") \
  .show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|  Month|    MonthlyRevenue|
+-------+------------------+
|2010-12| 833106.9300000004|
|2011-01| 691740.1900000015|
|2011-02|522940.69000000105|
|2011-03| 718575.5500000023|
|2011-04| 548662.0710000007|
|2011-05|  769400.950000001|
|2011-06| 768614.5900000014|
|2011-07| 718606.0710000012|
|2011-08| 760998.8500000011|
|2011-09| 1061252.453000001|
|2011-10|1170914.9299999995|
|2011-11|1528708.6899999976|
|2011-12| 474554.1100000001|
+-------+------------------+

Advanced Analysis

Top 10 Revenue-Generating Products Per Country

In [14]:
# Total revenue for each product in each country
df.groupBy("Country", "Description") \
  .agg(F.sum("Revenue").alias("TotalRevenue")) \
  .orderBy("Country", F.desc("TotalRevenue")) \
  .show(10, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------------------------------+------------------+
|Country  |Description                      |TotalRevenue      |
+---------+---------------------------------+------------------+
|Australia|RABBIT NIGHT LIGHT               |3375.84           |
|Australia|SET OF 6 SPICE TINS PANTRY DESIGN|2082.0            |
|Australia|RED TOADSTOOL LED NIGHT LIGHT    |1987.1999999999998|
|Australia|SET OF 3 CAKE TINS PANTRY DESIGN |1983.2            |
|Australia|REGENCY CAKESTAND 3 TIER         |1978.1999999999998|
|Australia|RED  HARMONICA IN BOX            |1810.8            |
|Australia|DOLLY GIRL LUNCH BOX             |1689.6            |
|Australia|MINI PAINT SET VINTAGE           |1630.8000000000002|
|Australia|SPACEBOY LUNCH BOX               |1583.9999999999998|
|Australia|RED RETROSPOT ROUND CAKE TINS    |1503.6            |
+---------+---------------------------------+------------------+
only showing top 10 rows

Average Quantity Purchased Per Product Per Customer

In [15]:
df.groupBy("CustomerID", "Description") \
  .agg(F.avg("Quantity").alias("AvgQuantity")) \
  .orderBy("CustomerID", F.desc("AvgQuantity")) \
  .show(10, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------------------------+-----------+
|CustomerID|Description                        |AvgQuantity|
+----------+-----------------------------------+-----------+
|12346     |MEDIUM CERAMIC TOP STORAGE JAR     |74215.0    |
|12347     |ICE CREAM SUNDAE LIP GLOSS         |240.0      |
|12347     |NAMASTE SWAGAT INCENSE             |36.0       |
|12347     |MINI PAINT SET VINTAGE             |36.0       |
|12347     |BLACK GRAND BAROQUE PHOTO FRAME    |30.0       |
|12347     |RABBIT NIGHT LIGHT                 |28.0       |
|12347     |3D DOG PICTURE PLAYING CARDS       |25.2       |
|12347     |BLUE NEW BAROQUE CANDLESTICK CANDLE|24.0       |
|12347     |PACK OF 60 SPACEBOY CAKE CASES     |24.0       |
|12347     |PINK NEW BAROQUECANDLESTICK CANDLE |24.0       |
+----------+-----------------------------------+-----------+
only showing top 10 rows

Monthly Revenue per Country

In [16]:
# Extract Month first
df = df.withColumn("Month", F.date_format("InvoiceTimestamp", "yyyy-MM"))

df.groupBy("Month", "Country") \
  .agg(F.sum("Revenue").alias("MonthlyRevenue")) \
  .orderBy("Month", "Country") \
  .show(20, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------------+------------------+
|Month  |Country        |MonthlyRevenue    |
+-------+---------------+------------------+
|2010-12|Australia      |1032.85           |
|2010-12|Austria        |277.20000000000005|
|2010-12|Bahrain        |205.74            |
|2010-12|Belgium        |1809.9099999999999|
|2010-12|Channel Islands|363.53            |
|2010-12|Cyprus         |1520.0200000000004|
|2010-12|Denmark        |1281.5            |
|2010-12|EIRE           |9958.860000000002 |
|2010-12|Finland        |1147.1999999999998|
|2010-12|France         |9591.489999999998 |
|2010-12|Germany        |15202.74          |
|2010-12|Iceland        |711.79            |
|2010-12|Italy          |811.4999999999999 |
|2010-12|Japan          |7633.31           |
|2010-12|Lithuania      |1661.0600000000002|
|2010-12|Netherlands    |8664.48           |
|2010-12|Norway         |3777.0399999999995|
|2010-12|Poland         |248.16            |
|2010-12|Portugal       |2439.9699999999993|
|2010-12|S

Top Customers per Country by Total Revenue

In [17]:
df.groupBy("Country", "CustomerID") \
  .agg(F.sum("Revenue").alias("CustomerRevenue")) \
  .orderBy("Country", F.desc("CustomerRevenue")) \
  .show(20, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------+------------------+
|Country  |CustomerID|CustomerRevenue   |
+---------+----------+------------------+
|Australia|12415     |123182.69         |
|Australia|12431     |5509.63           |
|Australia|12388     |2780.66           |
|Australia|12424     |1760.9600000000003|
|Australia|12393     |1622.2000000000003|
|Australia|12434     |806.1400000000001 |
|Australia|12386     |401.9             |
|Australia|12422     |386.20000000000005|
|Australia|16321     |373.65000000000003|
|Austria  |12360     |2657.0199999999995|
|Austria  |12865     |1568.2300000000002|
|Austria  |12818     |1542.0800000000004|
|Austria  |12358     |1168.06           |
|Austria  |12374     |742.93            |
|Austria  |12453     |707.09            |
|Austria  |12414     |562.41            |
|Austria  |12429     |437.98            |
|Austria  |12373     |364.6             |
|Austria  |12370     |277.2             |
|Austria  |12817     |166.04            |
+---------+----------+------------

Most Popular Products (by quantity) per Month

In [18]:
df.groupBy("Month", "Description") \
  .agg(F.sum("Quantity").alias("TotalQuantity")) \
  .orderBy("Month", F.desc("TotalQuantity")) \
  .show(20, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------------------------+-------------+
|Month  |Description                        |TotalQuantity|
+-------+-----------------------------------+-------------+
|2010-12|WORLD WAR 2 GLIDERS ASSTD DESIGNS  |5195         |
|2010-12|PACK OF 72 RETROSPOT CAKE CASES    |4115         |
|2010-12|WHITE HANGING HEART T-LIGHT HOLDER |3897         |
|2010-12|HAND WARMER BABUSHKA DESIGN        |3515         |
|2010-12|SMALL POPCORN HOLDER               |2760         |
|2010-12|MINI PAINT SET VINTAGE             |2710         |
|2010-12|PACK OF 12 LONDON TISSUES          |2698         |
|2010-12|GROW A FLYTRAP OR SUNFLOWER IN TIN |2617         |
|2010-12|PAPER CHAIN KIT 50'S CHRISTMAS     |2551         |
|2010-12|ASSORTED COLOUR BIRD ORNAMENT      |2403         |
|2010-12|ROTATING SILVER ANGELS T-LIGHT HLDR|2354         |
|2010-12|CREAM HEART CARD HOLDER            |2283         |
|2010-12|RED  HARMONICA IN BOX              |2230         |
|2010-12|ANTIQUE SILVER TEA GLASS ETCHED

Average Spending per Invoice by Country and Month

In [19]:
df.groupBy("Country", "Month", "InvoiceNo") \
  .agg(F.sum("Revenue").alias("InvoiceRevenue")) \
  .groupBy("Country", "Month") \
  .agg(F.avg("InvoiceRevenue").alias("AvgInvoiceSpending")) \
  .orderBy("Country", "Month") \
  .show(20, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------+------------------+
|Country  |Month  |AvgInvoiceSpending|
+---------+-------+------------------+
|Australia|2010-12|344.2833333333333 |
|Australia|2011-01|1000.8344444444444|
|Australia|2011-02|2877.8839999999996|
|Australia|2011-03|5571.330000000001 |
|Australia|2011-04|385.8             |
|Australia|2011-05|3180.6425000000004|
|Australia|2011-06|6296.942499999999 |
|Australia|2011-07|708.477142857143  |
|Australia|2011-08|11244.6           |
|Australia|2011-09|644.5662500000001 |
|Australia|2011-10|4287.6325         |
|Australia|2011-11|1207.1200000000001|
|Austria  |2010-12|277.20000000000005|
|Austria  |2011-02|259.18            |
|Austria  |2011-03|854.0600000000001 |
|Austria  |2011-04|340.39            |
|Austria  |2011-05|624.7149999999999 |
|Austria  |2011-07|595.975           |
|Austria  |2011-08|755.52            |
|Austria  |2011-10|1043.78           |
+---------+-------+------------------+
only showing top 20 rows

Customer Repeat Purchase Frequency per Product

In [20]:
df.groupBy("CustomerID", "Description") \
  .agg(F.countDistinct("InvoiceNo").alias("TimesPurchased")) \
  .orderBy(F.desc("TimesPurchased")) \
  .show(20, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------------------------+--------------+
|CustomerID|Description                       |TimesPurchased|
+----------+----------------------------------+--------------+
|16547     |DOTCOM POSTAGE                    |692           |
|16547     |Not Available                     |592           |
|16547     |JUMBO BAG RED RETROSPOT           |492           |
|16547     |JUMBO STORAGE BAG SUKI            |410           |
|16547     |JUMBO SHOPPER VINTAGE RED PAISLEY |385           |
|16547     |JUMBO BAG WOODLAND ANIMALS        |371           |
|16547     |JUMBO BAG PINK POLKADOT           |347           |
|16547     |RECYCLING BAG RETROSPOT           |341           |
|16547     |RED TOADSTOOL LED NIGHT LIGHT     |327           |
|16547     |SUKI  SHOULDER BAG                |326           |
|16547     |GREEN REGENCY TEACUP AND SAUCER   |324           |
|16547     |PARTY BUNTING                     |306           |
|16547     |WOODLAND CHARLOTTE BAG            |304     

Statistical Analysis

Average Revenue Per Invoice Per Country

In [21]:
df.groupBy("Country", "InvoiceNo") \
  .agg(F.sum("Revenue").alias("InvoiceRevenue")) \
  .groupBy("Country") \
  .agg(F.avg("InvoiceRevenue").alias("AvgInvoiceRevenue")) \
  .orderBy("AvgInvoiceRevenue", ascending=False) \
  .show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|             Country| AvgInvoiceRevenue|
+--------------------+------------------+
|           Singapore| 5627.851428571428|
|         Netherlands| 2991.846947368421|
|           Australia|2400.4215789473687|
|               Japan| 1931.147894736842|
|             Lebanon|           1693.88|
|           Hong Kong| 1464.790909090909|
|              Brazil|1143.6000000000001|
|              Sweden|1059.9047222222223|
|         Switzerland|1058.1403703703704|
|             Denmark|1057.7016666666668|
|              Norway|1044.6850000000002|
|              Israel|        1014.45125|
|                 RSA|1002.3099999999998|
|                EIRE|       991.8315625|
|              Greece| 938.9640000000002|
|              Cyprus| 862.3249999999999|
|     Channel Islands| 781.1707692307693|
|                 USA|           716.078|
|               Spain|  684.669111111111|
|United Arab Emirates| 625.6933333333333|
+--------------------+------------

Standard Deviation of Product Quantity Sold Per Country

In [22]:
df.groupBy("Country") \
  .agg(F.stddev("Quantity").alias("QuantityStdDev")) \
  .orderBy(F.desc("QuantityStdDev")) \
  .show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------+
|        Country|    QuantityStdDev|
+---------------+------------------+
|          Japan| 183.1718388970361|
|         Sweden| 128.7414424940944|
| United Kingdom|113.98801213789926|
|    Netherlands| 110.9341440260458|
|      Australia| 95.59736114829961|
|         Canada|47.292773943219196|
|           EIRE| 39.93297040222836|
|        Denmark|27.756898725528043|
|      Singapore| 27.56813180563665|
|        Bahrain|  25.8757715934338|
|         Cyprus|23.266366321494502|
|          Spain|22.764633800357654|
|Channel Islands|22.714635632560757|
|         Norway|22.632136039686564|
|        Finland| 22.42568730560871|
|        Austria|21.617731990474365|
|         France|21.071566895024585|
|        Iceland| 18.90814278875833|
|    Switzerland|18.508021755125537|
|        Germany|17.649191786900314|
+---------------+------------------+
only showing top 20 rows

Max and Min Revenue Per Product

In [23]:
df.groupBy("Description") \
  .agg(
    F.max("Revenue").alias("MaxRevenue"),
    F.min("Revenue").alias("MinRevenue")
  ) \
  .orderBy(F.desc("MaxRevenue")) \
  .show(10, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------+-----------------+----------+
|Description                        |MaxRevenue       |MinRevenue|
+-----------------------------------+-----------------+----------+
|MEDIUM CERAMIC TOP STORAGE JAR     |77183.6          |1.25      |
|PICNIC BASKET WICKER 60 PIECES     |38970.0          |649.5     |
|AMAZON FEE                         |13541.33         |219.76    |
|Adjust bad debt                    |11062.06         |11062.06  |
|POSTAGE                            |8142.75          |0.55      |
|SET OF TEA COFFEE SUGAR TINS PANTRY|7144.719999999999|4.95      |
|FAIRY CAKE FLANNEL ASSORTED COLOUR |6539.400000000001|0.79      |
|RABBIT NIGHT LIGHT                 |4992.0           |2.08      |
|WHITE HANGING HEART T-LIGHT HOLDER |4921.5           |0.0       |
|PAPER CHAIN KIT 50'S CHRISTMAS     |4781.6           |2.95      |
+-----------------------------------+-----------------+----------+
only showing top 10 rows

Average and Std. Dev of Unit Price Per Product

In [24]:
df.groupBy("Description") \
  .agg(
    F.avg("UnitPrice").alias("AvgPrice"),
    F.stddev("UnitPrice").alias("PriceStdDev")
  ) \
  .orderBy(F.desc("AvgPrice")) \
  .show(10, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------+------------------+------------------+
|Description                       |AvgPrice          |PriceStdDev       |
+----------------------------------+------------------+------------------+
|Adjust bad debt                   |11062.06          |null              |
|AMAZON FEE                        |6880.545          |9419.772483051276 |
|PICNIC BASKET WICKER 60 PIECES    |649.5             |0.0               |
|DOTCOM POSTAGE                    |291.31182203389835|355.56174416777293|
|Manual                            |248.43593052109182|661.8744086581926 |
|RUSTIC  SEVENTEEN DRAWER SIDEBOARD|158.07692307692307|9.703290480751681 |
|REGENCY MIRROR WITH SHUTTERS      |156.42857142857142|10.690449676496977|
|VINTAGE BLUE KITCHEN CABINET      |146.75            |66.03976075062658 |
|VINTAGE RED KITCHEN CABINET       |143.42105263157896|62.1365678028766  |
|CHEST NATURAL WOOD 20 DRAWERS     |118.07692307692308|7.783117824941562 |
+------------------------

Customer Purchase Behavior: Avg Revenue, Quantity, & Invoices

In [25]:
df.groupBy("CustomerID") \
  .agg(
    F.avg("Revenue").alias("AvgRevenue"),
    F.avg("Quantity").alias("AvgQuantity"),
    F.countDistinct("InvoiceNo").alias("TotalInvoices")
  ) \
  .orderBy(F.desc("AvgRevenue")) \
  .show(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+------------------+-------------+
|CustomerID|        AvgRevenue|       AvgQuantity|TotalInvoices|
+----------+------------------+------------------+-------------+
|     12346|           77183.6|           74215.0|            1|
|     15098|           13305.5|40.333333333333336|            3|
|     15749|           4453.43|            1802.8|            3|
|     15195|            3861.0|            1404.0|            1|
|     13135|            3096.0|            4300.0|            1|
|     17846|            2033.1|               1.0|            1|
|     18087|2027.8599999999997|            1953.5|            2|
|     16532|1687.1999999999998|             504.0|            3|
|     16000|1377.0777777777778| 567.7777777777778|            3|
|     16754|            1001.2|            2140.0|            1|
+----------+------------------+------------------+-------------+
only showing top 10 rows

Average Monthly Revenue Growth Per Country

In [26]:
monthly_rev = df.groupBy("Country", "Month") \
  .agg(F.sum("Revenue").alias("MonthlyRevenue"))

monthly_rev.groupBy("Country") \
  .agg(F.avg("MonthlyRevenue").alias("AvgMonthlyRevenue")) \
  .orderBy(F.desc("AvgMonthlyRevenue")) \
  .show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------+
|        Country| AvgMonthlyRevenue|
+---------------+------------------+
| United Kingdom| 685172.2957692313|
|           EIRE|21972.883846153847|
|    Netherlands|21863.496923076924|
|        Germany|17704.916923076926|
|         France| 16130.06923076923|
|      Australia|        11402.0025|
|      Singapore|           9848.74|
|    Switzerland| 4761.631666666667|
|          Spain| 4740.016923076923|
|         Norway|3760.8660000000004|
|          Japan|3669.1810000000005|
|        Belgium|3156.7853846153844|
|         Sweden|2935.1207692307685|
|       Portugal| 2603.242307692308|
|        Finland|           1891.29|
|      Hong Kong|            1790.3|
|        Denmark|1730.7845454545459|
|         Cyprus|1724.6500000000003|
|        Lebanon|1693.8799999999997|
|Channel Islands|1692.5366666666669|
+---------------+------------------+
only showing top 20 rows

Skew in Purchase Quantity Per Product (Min, Max, Avg)

In [27]:
df.groupBy("Description") \
  .agg(
    F.min("Quantity").alias("MinQty"),
    F.max("Quantity").alias("MaxQty"),
    F.avg("Quantity").alias("AvgQty")
  ) \
  .orderBy(F.desc("MaxQty")) \
  .show(10, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------+------+------+------------------+
|Description                        |MinQty|MaxQty|AvgQty            |
+-----------------------------------+------+------+------------------+
|MEDIUM CERAMIC TOP STORAGE JAR     |1     |74215 |304.87109375      |
|ASSTD DESIGN 3D PAPER STICKERS     |1     |12540 |326.54761904761904|
|Not Available                      |1     |5568  |54.97804054054054 |
|WORLD WAR 2 GLIDERS ASSTD DESIGNS  |1     |4800  |102.139146567718  |
|SMALL POPCORN HOLDER               |1     |4300  |33.2833607907743  |
|?                                  |101   |4000  |1352.0            |
|EMPIRE DESIGN ROSETTE              |1     |3906  |151.57142857142858|
|ESSENTIAL BALM 3.5g TIN IN ENVELOPE|1     |3186  |178.9090909090909 |
|FAIRY CAKE FLANNEL ASSORTED COLOUR |1     |3114  |38.215613382899626|
|came coded as 20713                |3100  |3100  |3100.0            |
+-----------------------------------+------+------+------------------+
only s

Variation in Spending Per Product Per Customer

In [28]:
df.groupBy("CustomerID", "Description") \
  .agg(
    F.avg("Revenue").alias("AvgSpent"),
    F.stddev("Revenue").alias("SpendStdDev")
  ) \
  .orderBy(F.desc("AvgSpent")) \
  .show(10, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------------------------+-----------------+------------------+
|CustomerID|Description                       |AvgSpent         |SpendStdDev       |
+----------+----------------------------------+-----------------+------------------+
|12346     |MEDIUM CERAMIC TOP STORAGE JAR    |77183.6          |null              |
|15098     |PICNIC BASKET WICKER 60 PIECES    |19809.75         |27096.685408459096|
|16547     |Adjust bad debt                   |11062.06         |null              |
|16029     |POSTAGE                           |8142.75          |null              |
|16547     |AMAZON FEE                        |6880.545         |9419.772483051276 |
|15749     |FAIRY CAKE FLANNEL ASSORTED COLOUR|6539.400000000001|0.0               |
|15749     |WHITE HANGING HEART T-LIGHT HOLDER|4776.75          |204.7074131535055 |
|15749     |DOORMAT FAIRY CAKE                |4388.5           |189.50461735799473|
|12536     |Manual                            |4161.06          |

RFM ANALYSIS

Calculate Recency

In [29]:
from pyspark.sql.functions import max, datediff, lit, to_date

# Step 1: Standardize InvoiceDate and convert to proper date
df = df.withColumn("InvoiceTimestamp", to_date("InvoiceTimestamp"))

# Step 2: Find the most recent date in the dataset
max_date = df.agg(max("InvoiceTimestamp")).collect()[0][0]

# Step 3: Recency = (most recent date in dataset) - (last purchase date per customer)
recency_df = df.groupBy("CustomerID") \
    .agg(max("InvoiceTimestamp").alias("LastPurchaseDate")) \
    .withColumn("Recency", datediff(lit(max_date), "LastPurchaseDate"))

recency_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------+-------+
|CustomerID|LastPurchaseDate|Recency|
+----------+----------------+-------+
|     12626|      2011-11-16|     23|
|     14423|      2011-05-05|    218|
|     17809|      2011-11-23|     16|
|     17708|      2011-06-05|    187|
|     17754|      2011-12-09|      0|
|     17783|      2011-09-28|     72|
|     16283|      2011-12-04|      5|
|     14837|      2011-09-11|     89|
|     17223|      2011-02-02|    310|
|     15535|      2011-06-19|    173|
|     14944|      2011-11-10|     29|
|     16232|      2011-10-27|     43|
|     16706|      2011-11-02|     37|
|     15429|      2011-11-13|     26|
|     16999|      2011-11-16|     23|
|     17346|      2011-12-06|      3|
|     16510|      2010-12-02|    372|
|     15594|      2011-11-24|     15|
|     16526|      2011-06-21|    171|
|     14030|      2011-11-21|     18|
+----------+----------------+-------+
only showing top 20 rows

Calculate Frequency

In [30]:
from pyspark.sql.functions import countDistinct

# Frequency = Count of unique invoices per customer
frequency_df = df.select("CustomerID", "InvoiceNo").dropDuplicates() \
    .groupBy("CustomerID") \
    .agg(countDistinct("InvoiceNo").alias("Frequency"))

frequency_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+
|CustomerID|Frequency|
+----------+---------+
|     16503|        4|
|     12626|        9|
|     17809|       12|
|     15535|        4|
|     15429|        2|
|     14944|       11|
|     16232|        4|
|     15594|        5|
|     17346|       15|
|     14030|        8|
|     16766|        2|
|     12723|        5|
|     18014|        1|
|     17926|        2|
|     15491|        6|
|     13751|        1|
|     13994|        6|
|     16274|        1|
|     12494|        7|
|     16671|        3|
+----------+---------+
only showing top 20 rows

Calculate Monetary

In [31]:
from pyspark.sql.functions import sum, round, col

# Step 1: Create a Revenue column per transaction
df = df.withColumn("Revenue", col("Quantity") * col("UnitPrice"))

# Step 2: Monetary = Total Revenue per Customer
monetary_df = df.groupBy("CustomerID") \
    .agg(round(sum("Revenue"), 2).alias("Monetary"))

monetary_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+
|CustomerID|Monetary|
+----------+--------+
|     16339|   114.3|
|     16503| 1431.93|
|     12626| 6620.48|
|     17783|  275.48|
|     17809| 5162.91|
|     16283| 1616.62|
|     14837|  1649.5|
|     17708|  267.08|
|     17754| 1857.46|
|     13483|  140.34|
|     17223|  442.04|
|     13508|  110.97|
|     15535|   549.3|
|     16706|   549.2|
|     14660|  285.74|
|     14944| 5865.57|
|     13504|  295.93|
|     15429|  414.55|
|     17346| 2717.71|
|     15594| 1771.78|
+----------+--------+
only showing top 20 rows

In [32]:
# Join Recency, Frequency, and Monetary on CustomerID
rfm_df = recency_df.join(frequency_df, "CustomerID") \
                   .join(monetary_df, "CustomerID")
rfm_df = rfm_df.drop("LastPurchaseDate")
rfm_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+---------+--------+
|CustomerID|Recency|Frequency|Monetary|
+----------+-------+---------+--------+
|     16339|    284|        1|   114.3|
|     12626|     23|        9| 6620.48|
|     17809|     16|       12| 5162.91|
|     16283|      5|        5| 1616.62|
|     15429|     26|        2|  414.55|
|     16999|     23|        2|  484.82|
|     14944|     29|       11| 5865.57|
|     17346|      3|       15| 2717.71|
|     14030|     18|        8| 2344.22|
|     17396|     39|        8|  7330.8|
|     16801|    163|        4| 1606.98|
|     13994|      3|        6| 2362.26|
|     16671|     28|        3| 1692.27|
|     13726|     38|        2| 1079.66|
|     13104|      3|        6| 1098.48|
|     15062|     71|        4|  987.86|
|     12705|     15|        9| 6863.25|
|     17895|     44|        1|    98.0|
|     14961|     10|        9| 7024.88|
|     16431|    213|        3|  749.48|
+----------+-------+---------+--------+
only showing top 20 rows

In [33]:
rfm_df.write \
    .option("header", "true") \
    .mode("overwrite") \
    .csv("s3://project-tarun/Output/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…